In [ ]:
import os
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.interpolate import interp1d
from ahrs.filters import Madgwick

In [ ]:
# Change root directory
dirPath = r'C:\Users\Andreas\Desktop\Thesis\Data'
os.chdir(dirPath)
print('Current working directory: {0}'.format(os.getcwd()))

In [ ]:
def read_csv_files(dirPath = dirPath, sample = 'all', exercise = 'all'):
    root = dirPath+'\\'+'Labelled'
    df = pd.DataFrame()

    for samplefolder in os.listdir(root):
        if sample in samplefolder or sample == 'all':

            for exercisefolder in os.listdir(root+'\\'+samplefolder):
                if exercise in exercisefolder or exercise == 'all':
                    
                    for file in os.listdir(root+'\\'+samplefolder+'\\'+exercisefolder):
                        if '.csv' in file:
                            csv_path = root+'\\'+samplefolder+'\\'+exercisefolder+'\\'+file
                            data = pd.read_csv(csv_path, delimiter=',', encoding = 'utf-8')
                            df = pd.concat([df, data], ignore_index=True)
    return df

In [ ]:
def read_csv(csv, dirPath = dirPath):
                    
    for file in os.listdir(dirPath):
        if csv in file:
            new_df = pd.read_csv(file, delimiter=',', encoding = 'utf-8')
    return new_df

In [ ]:
def restructure_df(df):

    csv_df = df.copy()
    new_df = pd.DataFrame()
    # Drop video timestamp
    csv_df.drop(columns='time', inplace = True)
    csv_df.drop(csv_df[csv_df.SensorName == 'video'].index, inplace = True)
    # Drop not labelled data
    csv_df.drop(csv_df[csv_df.label == 'Unassigned'].index, inplace = True)
    csv_df.drop(csv_df[csv_df.label == 'idle'].index, inplace = True)
    
    # Group by recording
    groups = csv_df.groupby(['Subject','Exercise'])
    for _, dfGroup in groups:
        
        data = pd.DataFrame()
        i=0

        merge_groups = dfGroup.groupby(['DataType','SensorName'])
        for [strDatatype,strSensorName], dfDatatype in merge_groups:

            dfDatatype.reset_index(inplace=True,drop=True)

            # Create new columns for different sensors
            for _,axis in enumerate(['x', 'y', 'z']):
                col_name = f'{strSensorName}_{strDatatype}_{axis}'
                dfDatatype[col_name] = dfDatatype[axis]
            
            # Drop unnecesary columns
            dfDatatype.drop(columns=['x','y','z'], inplace = True)

            # First iteration
            if i == 0:
                data = dfDatatype
                i+=1
            else:
                # Difference between different sensor timestamps
                diff = np.abs(data.t.values.reshape(-1, 1) - dfDatatype.t.values)
                # Index of the nearest t
                nearest = diff.argmin(axis=0)
                dfDatatype.t = data.t.iloc[nearest].values
                
                for col in dfDatatype.columns.values: 
                    if col not in data.columns.values:
                        data[col] = dfDatatype[col]

        # Drop possible extra timestamp indices
        data.dropna(inplace=True)

        # Reset time for each rep
        for _,rep in enumerate(data.rep.unique()):
            # Find rep start
            rep_start = data.t.loc[(data.rep == rep).argmax()]
            # Assign difference to list
            t = []
            for value in data.t.loc[(data.rep == rep)]:
                t.append(value-rep_start)
            # Change value
            data.loc[(data.rep == rep), 't'] = np.around(np.array(t),2)
        
        new_df = pd.concat([new_df, data],ignore_index=True)
    
    # Replace partitioned exercise
    new_df['Exercise'] = new_df['Exercise'].str.replace(' 1', '').str.replace(' 2', '')
    # Drop reduntant columns
    new_df.drop(columns=['DataType','SensorName'], inplace = True)
    # Drop miscalculated timestamps
    new_df = new_df.loc[new_df.t < 4]
    #Transform rep to string
    new_df['rep'] = new_df['rep'].astype(str)
    # Drop rows with missing sensor data
    new_df.dropna(inplace=True)

    new_df.to_csv('raw_dataframe_'+time.strftime("%Y%m%d_%H%M")+'.csv', sep=',', index=False)

    return new_df

In [ ]:
def remove_outliers(data):

    clean_df = pd.DataFrame()
    dirty_df = pd.DataFrame()

    # Group by recording
    groups = data.groupby(['Subject','Exercise'])
    for strGroup, dfGroup in groups:

        rep_groups = dfGroup.groupby('rep')
        for strRep, dfRep in rep_groups:

            tmean = dfRep['t'].mean()
            tstd = dfRep['t'].std()
            threshold = 3 * tstd
            dfRep = dfRep[(dfRep['t'] - tmean).abs() < threshold]

            for col in dfRep.iloc[:,5:]:
                colmean = dfRep[col].mean()
                colstd = dfRep[col].std()
                threshold = 3*colstd
                # cutRep = dfRep[(dfRep[col] - colmean).abs() > threshold]
                cleanRep = dfRep[(dfRep[col] - colmean).abs() < threshold]

            clean_df = pd.concat([clean_df,cleanRep],ignore_index=True)
            # dirty_df = pd.concat([dirty_df,cutRep],ignore_index=True)        
    
    clean_df.dropna()
    clean_df.to_csv('clean_dataframe_'+time.strftime("%Y%m%d_%H%M")+'.csv', sep=',', index=False)

    return clean_df

In [ ]:
def resample(data):

    resampled_df = pd.DataFrame()

    # Group by recording
    groups = data.groupby(['Subject','Exercise'])
    for strGroup, dfGroup in groups:

        rep_groups = dfGroup.groupby('rep')
        for strRep, dfRep in rep_groups:

            #Target time window
            sample_window = 3
            # Target frequency
            sample_f = 50
            # Desired number of samples
            num_samples = sample_window * sample_f

            dfRep.reset_index(inplace=True,drop=True)
            resampled = pd.DataFrame()
            for col in dfRep.iloc[:,5:]:

                x_old = np.linspace(0, 1, len(dfRep[col]))
                f = interp1d(x_old, dfRep[col], kind='linear')
                
                x_new = np.linspace(0, 1, num_samples)
                resampled[col] = f(x_new)

            resampled.insert(0,'Subject', strGroup[0])
            resampled.insert(1,'Exercise', strGroup[1])
            resampled.insert(2,'rep', strRep)
            resampled.insert(3,'t', np.linspace(0, sample_window, num_samples))
            resampled.insert(4,'label', dfRep.loc[0,'label'])

            resampled_df = pd.concat([resampled_df,resampled],ignore_index=True)
            
    resampled_df.to_csv('resampled_dataframe_'+time.strftime("%Y%m%d_%H%M")+'.csv', sep=',', index=False)

    return resampled_df

In [ ]:
def butterworth_filter(data, cutoff_frequency, fs, order):
    
    nyquist_frequency = 0.5 * fs

    # Group by recording
    groups = data.groupby(['Subject','Exercise'])
    for strGroup, dfGroup in groups:
        # Group by rep
        rep_groups = dfGroup.groupby('rep')
        for strRep, dfRep in rep_groups:
            dfRep.reset_index(inplace=True,drop=True)

            for col in dfRep.iloc[:,5:]:

                if 'Acc' in col:
                    normalized_cutoff_frequency = cutoff_frequency[0] / nyquist_frequency
                    filter_type = 'low'
                    b, a = signal.butter(order, normalized_cutoff_frequency, btype=filter_type, analog=False)

                    data.loc[(data.Subject == strGroup[0]) & (data.Exercise == strGroup[1]) & (data.rep == strRep), col] = signal.filtfilt(b, a, dfRep[col])

                elif 'Gyro' in col:
                    normalized_cutoff_frequency = cutoff_frequency[1] / nyquist_frequency
                    filter_type = 'high'
                    b1, a1 = signal.butter(order, normalized_cutoff_frequency, btype=filter_type, analog=False)
                    
                    normalized_cutoff_frequency = cutoff_frequency[0] / nyquist_frequency
                    filter_type = 'low'
                    b2, a2 = signal.butter(order, normalized_cutoff_frequency, btype=filter_type, analog=False)

                    data.loc[(data.Subject == strGroup[0]) & (data.Exercise == strGroup[1]) & (data.rep == strRep), col] = signal.filtfilt(b1, a1, dfRep[col])
                    data.loc[(data.Subject == strGroup[0]) & (data.Exercise == strGroup[1]) &  (data.rep == strRep), col] = signal.filtfilt(b2, a2, dfRep[col])

    data.to_csv('butter_dataframe_'+time.strftime("%Y%m%d_%H%M")+'.csv', sep=',', index=False)
    return data

In [ ]:
def complementary_filter(data):
    
    comp_df = pd.DataFrame()

    # Group by recording
    groups = data.groupby(['Subject','Exercise'])
    for strGroup, dfGroup in groups:
        # Group by rep
        rep_groups = dfGroup.groupby('rep')
        for strRep, dfRep in rep_groups:
            dfRep.reset_index(inplace=True,drop=True)

            result = pd.DataFrame()
            df = dfRep.iloc[:,5:]
            
            for col in df.columns:
    
                sensor_name, _, _ = col.split('_')

                acc = np.array([df[sensor_name + '_Acc_x'], df[sensor_name + '_Acc_y'], df[sensor_name + '_Acc_z']])
                gyro = np.array([df[sensor_name + '_Gyro_x'], df[sensor_name + '_Gyro_y'], df[sensor_name + '_Gyro_z']])

                madgwick = Madgwick(gyr=gyro.T, acc=acc.T, frequency=50)
                
                # Assign quaternions
                for q in range(madgwick.Q.shape[1]):
                    result[sensor_name + f'_q{q}'] = madgwick.Q[:,q]
            
            result.insert(0,'Subject', strGroup[0])
            result.insert(1,'Exercise', strGroup[1])
            result.insert(2,'rep', strRep)
            result.insert(3,'t', dfRep['t'])
            result.insert(4,'label', dfRep.loc[0,'label'])
             
            comp_df = pd.concat([comp_df, result], ignore_index=True)
    
    comp_df.to_csv('complementary_dataframe_'+time.strftime("%Y%m%d_%H%M")+'.csv', sep=',', index=False)
    return comp_df

In [ ]:
def plot_datapoints(data):

    # Group by recording
    groups = data.groupby(['Exercise','Subject'])
    for strGroup, dfGroup in groups:
        print(strGroup)
        fig, ax = plt.subplots(nrows=1, ncols=3, layout="constrained", figsize=(30,10))
        for col in dfGroup.iloc[:,5:]:
            if 'Acc' in col:
                if '_x' in col:
                    j=0
                elif '_y' in col:
                    j=1
                elif '_z' in col:
                    j=2
                dfGroup.sort_values(by = 't')
                ax[j].scatter(dfGroup['t'], dfGroup[col], marker = '.', s = 10 , label = col)
                ax[j].legend()
        plt.show()

In [ ]:
def plot_filters(raw, resampled, butter):

    fig, ax = plt.subplots(nrows=1, ncols=2, layout="constrained", figsize=(15,6))
    for df in [raw, resampled, butter]:
        acc = df.loc[(df.label == 'Correct repetition')&(df.rep == 17)&(df.Exercise == 'Squats')&(df.Subject == 'Subject_6'), ['t','Lumbar_Acc_z']]
        gyro = df.loc[(df.label == 'Correct repetition')&(df.rep == 17)&(df.Exercise == 'Squats')&(df.Subject == 'Subject_6'), ['t','Lumbar_Gyro_z']]

        ax[0].plot(acc['t'], acc['Lumbar_Acc_z'])
        ax[1].plot(gyro['t'], gyro['Lumbar_Gyro_z'])
        
        ax[0].set_title('Accelerometer filtering sample')
        ax[0].set_ylabel('Acceleration (g)')
        ax[0].set_xlabel('Time (s)')

        ax[1].set_title('Gyroscope filtering sample')
        ax[1].set_ylabel('Angular velocity (rad/s)')
        ax[1].set_xlabel('Time (s)')

        ax[0].legend(['Raw data', 'Resampled data', 'Butterworth filter'], loc='upper right')
        ax[1].legend(['Raw data', 'Resampled data', 'Butterworth filter'], loc='upper right')
    plt.show()

In [ ]:
def plot_quaternions(data):
    # Group by recording
    groups = data.groupby(['Subject','Exercise','rep'])
    for strGroup, dfGroup in groups:
        if strGroup[1] == 'Lunges' and strGroup[2] == 7:
            fig, ax = plt.subplots(nrows=1, ncols=4, layout="constrained", figsize=(30,6))
            for col in dfGroup.iloc[:,5:]:
                    if 'q0' in col:
                        ax[0].plot(dfGroup['t'], dfGroup[col], label = col)
                        ax[0].set_ylabel('q0')
                        ax[0].set_xlabel('time (s)')
                        ax[0].set_title("Quaternion 0")
                    elif 'q1' in col:
                        ax[1].plot(dfGroup['t'], dfGroup[col], label = col)
                        ax[1].set_ylabel('q1')
                        ax[1].set_xlabel('time (s)')
                        ax[1].set_title("Quaternion 1")
                    elif 'q2' in col:
                        ax[2].plot(dfGroup['t'], dfGroup[col], label = col)
                        ax[2].set_ylabel('q2')
                        ax[2].set_xlabel('time (s)')
                        ax[2].set_title("Quaternion 2")
                    elif 'q3' in col:
                        ax[3].plot(dfGroup['t'], dfGroup[col], label = col)
                        ax[3].set_ylabel('q3')
                        ax[3].set_xlabel('time (s)')
                        ax[3].set_title("Quaternion 3")
                        
            plt.show()

In [ ]:
def plot_angles_correlation(df, exercise = 'all'):

    if exercise in df.Exercise.values:
        df = df.groupby('Exercise').get_group(exercise)
    
    df = df.iloc[:,5:]
    # Calculate correlation matrix
    corr_matrix = df.corr()
    # Plot the correlation matrix as a heatmap
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax = 1)

    # Add the correlation values to the heatmap
    for i in range(len(corr_matrix)):
        for j in range(len(corr_matrix)):
            ax.text(j, i, round(corr_matrix.iloc[i, j], 2),
                        ha="center", va="center", color="black")

    # Set the axis labels
    ax.set_xticks(range(len(df.columns)))
    ax.set_yticks(range(len(df.columns)))
    ax.set_xticklabels(df.columns)
    ax.set_yticklabels(df.columns)

    # Rotate the x-axis labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
            rotation_mode="anchor")

    # Add a title
    ax.set_title(f'Quaternions - {exercise}')

    plt.show()

In [ ]:
def plot_correlation(df, exercise = 'all'):

    if exercise in df.Exercise.values:
        df = df.groupby('Exercise').get_group(exercise)
    
    df = df.iloc[:,5:]
    corrdf = pd.DataFrame()

    for col in df.columns:
        sensor_name, sensor_type,_ = col.split('_')
        # Find magnitude to determine correlation (instead of using each axis)
        magnitude = np.sqrt(df[sensor_name + '_' + sensor_type + '_' + 'x']**2 +
                            df[sensor_name + '_' + sensor_type + '_' + 'y']**2 +
                            df[sensor_name + '_' + sensor_type + '_' + 'z']**2)
        
        corrdf[sensor_name + '_' + sensor_type] = magnitude
    
    # Calculate correlation matrix
    corr_matrix = corrdf.corr()
    # Plot the correlation matrix as a heatmap
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax = 1)

    # Add the correlation values to the heatmap
    for i in range(len(corr_matrix)):
        for j in range(len(corr_matrix)):
            ax.text(j, i, round(corr_matrix.iloc[i, j], 2),
                        ha="center", va="center", color="black")

    # Set the axis labels
    ax.set_xticks(range(len(corrdf.columns)))
    ax.set_yticks(range(len(corrdf.columns)))
    ax.set_xticklabels(corrdf.columns)
    ax.set_yticklabels(corrdf.columns)

    # Rotate the x-axis labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
            rotation_mode="anchor")

    # Add a title
    ax.set_title(f'Resampled signal - {exercise}')

    plt.show()

In [ ]:
def plot_metrics_correlation(df, exercise = 'all', sensor = 'all'):

    if exercise in df.Exercise.values:
        df = df.groupby('Exercise').get_group(exercise)
        
    if df.columns.str.contains(sensor).any():
        df = df.filter(like=sensor)
    
    df = df.iloc[:,4:]
    corrdf = pd.DataFrame()

    for col in df.columns:
        sensor_name, sensor_type, axis, metric = col.split('_')
        
        if axis in ['x','y','z']:
            magnitude = np.sqrt(df['Lumbar_' + sensor_type + '_' + 'x'+ '_' + metric]**2 +
                                df['Lumbar_' + sensor_type + '_' + 'y'+ '_' + metric]**2 +
                                df['Lumbar_' + sensor_type + '_' + 'z'+ '_' + metric]**2)
            corrdf['Lumbar_'+sensor_type+ '_' + metric] = magnitude

    corr_matrix = corrdf.corr()
    # Plot the correlation matrix as a heatmap
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax = 1)

    # Add the correlation values to the heatmap
    for i in range(len(corr_matrix)):
        for j in range(len(corr_matrix)):
            ax.text(j, i, round(corr_matrix.iloc[i, j], 2),
                        ha="center", va="center", color="black")

    # Set the axis labels
    ax.set_xticks(range(len(corrdf.columns)))
    ax.set_yticks(range(len(corrdf.columns)))
    ax.set_xticklabels(corrdf.columns)
    ax.set_yticklabels(corrdf.columns)

    # Rotate the x-axis labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
            rotation_mode="anchor")

    # Add a title
    ax.set_title(f'Statistical features - {exercise}')

    plt.show()

In [ ]:
def statistical_features(df,name):

    sdf = pd.DataFrame()

    # wrlds "normalization"
#     for col in df.columns:
#          if 'Acc' in col:
#               df[col] = df[col]/16
#          elif 'Gyro' in col:
#               df[col] = df[col]/2000
    
    # Group by recording
    groups = df.groupby(['Subject','Exercise'])
    for strGroup, dfGroup in groups:
     rep_groups = dfGroup.groupby('rep')
     for strRep, dfRep in rep_groups:
          dfRep.reset_index(inplace=True,drop=True)

          maxdf = pd.DataFrame(dfRep.iloc[:,5:].max().add_suffix('_max')).T
          mindf = pd.DataFrame(dfRep.iloc[:,5:].min().add_suffix('_min')).T
          mediandf = pd.DataFrame(dfRep.iloc[:,5:].median().add_suffix('_median')).T
          meandf = pd.DataFrame(dfRep.iloc[:,5:].mean().add_suffix('_mean')).T
          stddf = pd.DataFrame(dfRep.iloc[:,5:].std().add_suffix('_std')).T
          kurtdf = pd.DataFrame(dfRep.iloc[:,5:].kurtosis().add_suffix('_kurtosis')).T
          skewdf = pd.DataFrame(dfRep.iloc[:,5:].skew().add_suffix('_skewness')).T

          repdf = pd.concat([maxdf,mindf,mediandf,
                              meandf,stddf,
                              kurtdf,skewdf],
                              axis = 1)
          repdf.insert(0,'Subject', strGroup[0])
          repdf.insert(1,'Exercise', strGroup[1])
          repdf.insert(2,'label', dfRep.loc[0,'label'])

          sdf = pd.concat([sdf, repdf],ignore_index=True)

    sdf.to_csv(name+time.strftime("%Y%m%d_%H%M")+'.csv', sep=',', index=False)
    return sdf

In [ ]:
def features(df,name):

    asdf = pd.DataFrame()
    
    # Group by recording
    groups = df.groupby(['Subject','Exercise'])
    for strGroup, dfGroup in groups:

        rep_groups = dfGroup.groupby('rep')
        for strRep, dfRep in rep_groups:
            dfRep.reset_index(inplace=True,drop=True)

            repdf = pd.DataFrame(dfRep.loc[dfRep.index[0]]).T

            for col in dfRep.iloc[:,5:]:
                repdf.at[0, col] = dfRep[col].tolist()
   
            asdf = pd.concat([asdf, repdf],ignore_index=True)
    
    
    asdf.drop(columns=['rep','t'], inplace = True)
    asdf.to_csv(name+time.strftime("%Y%m%d_%H%M")+'.csv', sep=',', index=False)
    return asdf

In [ ]:
df = read_csv_files()
new_df = restructure_df(df)
# clean_df = remove_outliers(new_df)
# resampled_df = resample(clean_df)
plot_datapoints(new_df)

In [ ]:
# #Butterworth
resampled_df = read_csv('resampled_dataframe_')

# Parameters
cutoff_frequency = [3, 0.5]
fs = 50
order = 2

butter_df = butterworth_filter(resampled_df, cutoff_frequency, fs, order)

In [ ]:
raw = read_csv('clean_dataframe')
resampled = read_csv('resampled_dataframe')
butter = read_csv('butter_dataframe')
plot_filters(raw, resampled, butter)

In [ ]:
butter = read_csv('butter_dataframe')
compl = complementary_filter(butter)
plot_quaternions(compl)

In [ ]:
raw = read_csv('raw_dataframe')
resampled = read_csv('resampled_dataframe')
butter = read_csv('butter_dataframe')

raw_stats = statistical_features(raw,'raw_statistical_features_')
res_stats = statistical_features(resampled,'resampled_statistical_features_')
filt_stats = statistical_features(butter,'filtered_statistical_features_')

In [ ]:
resampled = read_csv('resampled_dataframe')
butter = read_csv('butter_dataframe')
compl = read_csv('complementary_dataframe')

rdf = features(resampled,'resampled_features_')
fdf = features(butter,'butter_features_')
adf = features(compl,'angles_features_')

In [ ]:
resampled = read_csv('resampled_dataframe_')
compl = read_csv('complementary_dataframe_')
stats = read_csv('statistical_features_')

plot_correlation(resampled)
plot_angles_correlation(compl)
plot_metrics_correlation(stats)